## Model selection

Build ML pipelines and through cross-validation and assessment choose a model that performs the best

### Import libraries

In [1]:
# print_function for compatibility with Python 3
from __future__ import print_function
# NumPy for numerical computing
import numpy as np
# Pandas for DataFrames
import pandas as pd
# Matplotlib for visualization
from matplotlib import pyplot as plt
# display plots in the notebook
%matplotlib inline
# Seaborn for easier visualization
import seaborn as sns
# Scikit-Learn for Modeling
import sklearn
# Pickle for saving model files
import pickle

In [2]:
# Import Logistic Regression
from sklearn.linear_model import LogisticRegression
# Import RandomForestClassifier and GradientBoostingClassifer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
# Function for splitting training and test set
from sklearn.model_selection import train_test_split
# Function for creating model pipelines
from sklearn.pipeline import make_pipeline
# For standardization
from sklearn.preprocessing import StandardScaler
# Helper for cross-validation
from sklearn.model_selection import GridSearchCV

In [23]:
# Classification metrics (added later)
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

### Load datasets

In [5]:
abt_red = pd.read_csv('abt_red.csv')
abt_white = pd.read_csv('abt_white.csv')

### Split dataset

It is important to stratify our sampling because we have many more mediocre observations than good or bad ones.

In [6]:
red_quality = abt_red.quality
red_input_features = abt_red.drop('quality', axis=1)
red_input_train, red_input_test, red_quality_train, red_quality_test = train_test_split(
    red_input_features, red_quality, test_size=0.2, random_state=123, stratify=red_quality)

white_quality = abt_white.quality
white_input_features = abt_white.drop('quality', axis=1)
white_input_train, white_input_test, white_quality_train, white_quality_test = train_test_split(
    white_input_features, white_quality, test_size=0.2, random_state=123, stratify=white_quality)

In [7]:
## Make sure the 'quality' column is not present
red_input_features.head()

,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


In [8]:
## Make sure the 'quality' column is not present
white_input_features.head()

,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8
1,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5
2,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1
3,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9
4,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9


In [9]:
## Check the size of train/test splits for red wine
print(red_quality_train.shape, red_input_train.shape)
print(red_quality_test.shape, red_input_test.shape)

(1279,) (1279, 10)
(320,) (320, 10)


In [10]:
## Check the size of train/test splits for white wine
print(white_quality_train.shape, white_input_train.shape)
print(white_quality_test.shape, white_input_test.shape)

(3918,) (3918, 10)
(980,) (980, 10)


### Create pipelines and declare hyperparameters

In [38]:
# Pipeline dictionary
pipelines = {
    'l1': make_pipeline(StandardScaler(), LogisticRegression(random_state=123, penalty='l1')),
    'l2': make_pipeline(StandardScaler(), LogisticRegression(random_state=123, penalty='l2')),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier(random_state=123)),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=123))
}

# Hyperparameters to tune
hyperparameters = {
    'l1': {
        'logisticregression__C': np.linspace(1e-3, 1e3, 10)
    },
    'l2': {
        'logisticregression__C': np.linspace(1e-3, 1e3, 10)
    },
    'rf': {
        'randomforestclassifier__n_estimators': [100, 200],
        'randomforestclassifier__max_features': ['auto', 'sqrt', 0.33]
    },
    'gb': {
        'gradientboostingclassifier__n_estimators': [100, 200],
        'gradientboostingclassifier__learning_rate': [0.05, 0.1, 0.2],
        'gradientboostingclassifier__max_depth': [1,3,5]
    }
}

### Run cross-validation loops and train models

In [39]:
def fit_models(X_train, y_train):
    # Create empty dictionary called fitted_models
    fitted_models = {}

    # Loop through model pipelines, tuning each one and saving it to fitted_models
    for pipeline in pipelines.keys():

        # Create cross-validation object from pipeline and hyperparameters
        model = GridSearchCV(pipelines[pipeline], hyperparameters[pipeline], cv=10, n_jobs=-1)

        # Fit model on X_train, y_train
        model.fit(X_train, y_train)

        # Store model in fitted_models[name] 
        fitted_models[pipeline] = model

        # Print '{name} has been fitted'
        print('{} has been fitted'.format(pipeline))
        
    return fitted_models

In [40]:
fitted_red_models = fit_models(red_input_train, red_quality_train)

l1 has been fitted
l2 has been fitted
rf has been fitted
gb has been fitted


In [41]:
fitted_white_models = fit_models(white_input_train, white_quality_train)

l1 has been fitted
l2 has been fitted
rf has been fitted
gb has been fitted


### Evaluate performance and choose the winner

Higher R^2 score and lower MAE is better.

In [42]:
## Display best R^2 holdout score (red)
for name, model in fitted_red_models.items():
    print(name, model.best_score_)

l1 0.842064112588
l2 0.842064112588
rf 0.87255668491
gb 0.869429241595


In [43]:
## Display best R^2 holdout score (red)
for name, model in fitted_white_models.items():
    print(name, model.best_score_)

l1 0.768504338948
l2 0.768504338948
rf 0.839714139867
gb 0.828739152629


In [44]:
def evaluate_perf(fitted_models, X_test, y_test):
    # Predict classes using L1-regularized logistic regression 
    for model in fitted_models.keys():
        y_pred = fitted_models[model].predict(X_test)
        cm = confusion_matrix(y_test, y_pred)
        print(model, accuracy_score(y_test, y_pred))
        print(cm)
        print(classification_report(y_test, y_pred))

In [45]:
## Evaluate performance of score prediction for reds
evaluate_perf(fitted_red_models, red_input_test, red_quality_test)

l1 0.828125
[[  0  13   0]
 [  0 256   8]
 [  0  34   9]]
             precision    recall  f1-score   support

          0       0.00      0.00      0.00        13
          1       0.84      0.97      0.90       264
          2       0.53      0.21      0.30        43

avg / total       0.77      0.83      0.79       320

l2 0.828125
[[  0  13   0]
 [  0 256   8]
 [  0  34   9]]
             precision    recall  f1-score   support

          0       0.00      0.00      0.00        13
          1       0.84      0.97      0.90       264
          2       0.53      0.21      0.30        43

avg / total       0.77      0.83      0.79       320

rf 0.859375
[[  1  12   0]
 [  0 257   7]
 [  0  26  17]]
             precision    recall  f1-score   support

          0       1.00      0.08      0.14        13
          1       0.87      0.97      0.92       264
          2       0.71      0.40      0.51        43

avg / total       0.85      0.86      0.83       320

gb 0.85625
[[  1  12  

/home/sergey/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [46]:
## Evaluate performance of score prediction for whites
evaluate_perf(fitted_red_models, white_input_test, white_quality_test)

l1 0.752040816327
[[  0  36   1]
 [  0 706  25]
 [  0 181  31]]
             precision    recall  f1-score   support

          0       0.00      0.00      0.00        37
          1       0.76      0.97      0.85       731
          2       0.54      0.15      0.23       212

avg / total       0.69      0.75      0.69       980

l2 0.752040816327
[[  0  36   1]
 [  0 706  25]
 [  0 181  31]]
             precision    recall  f1-score   support

          0       0.00      0.00      0.00        37
          1       0.76      0.97      0.85       731
          2       0.54      0.15      0.23       212

avg / total       0.69      0.75      0.69       980

rf 0.745918367347
[[  0  35   2]
 [  0 695  36]
 [  0 176  36]]
             precision    recall  f1-score   support

          0       0.00      0.00      0.00        37
          1       0.77      0.95      0.85       731
          2       0.49      0.17      0.25       212

avg / total       0.68      0.75      0.69       980

gb 0

/home/sergey/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Result

According to the confusion matrices and the accuracy precision metric all of the models suffer from inability to accurately predict the bad and excellent wines. The bad wines are especially difficult to predict. All models have failed at it.